# RandomMachine Advanced Guide

This notebook provides advanced examples and best practices for using the RandomMachine library.

## Contents

1. [Understanding Random Boosting](#understanding-random-boosting)
2. [Custom Model Configuration](#custom-model-configuration)
3. [Performance Optimization](#performance-optimization)
4. [Real-World Examples](#real-world-examples)
5. [Troubleshooting](#troubleshooting)

## 1. Understanding Random Boosting

RandomMachine implements Newton boosting with random base learner selection. At each iteration:

1. Compute gradients and Hessians of the loss function
2. **Randomly select** a base learner from the pool
3. Fit the base learner to the negative gradient
4. Add weighted predictions to the ensemble

This random selection provides:
- **Diversity**: Different model types in the ensemble
- **Robustness**: Less prone to overfitting on specific patterns
- **Flexibility**: Can combine strengths of different algorithms

In [ ]:
# Setup
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression, make_classification, load_boston, load_iris
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, accuracy_score

from randommachine import (
    RandomLGBMRegressor, RandomCatBoostRegressor, 
    RandomRegressor, HeteroBoostRegressor,
    RandomLGBMClassifier, RandomCatBoostClassifier,
    MeanSquaredError, LogisticLoss
)

np.random.seed(42)
print("Setup complete!")

## 2. Custom Model Configuration

### Configuring Tree Depths

The `min_max_depth` and `max_max_depth` parameters control the pool of tree depths to sample from.

In [ ]:
# Example: Different depth configurations
configs = [
    {'min': 2, 'max': 3, 'name': 'Shallow Trees'},
    {'min': 4, 'max': 6, 'name': 'Medium Trees'},
    {'min': 7, 'max': 10, 'name': 'Deep Trees'},
]

X, y = make_regression(n_samples=500, n_features=15, noise=10, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

results = {}

for config in configs:
    model = RandomLGBMRegressor(
        num_iterations=15,
        learning_rate=0.5,
        min_max_depth=config['min'],
        max_max_depth=config['max'],
        tree_iterations=50,
        random_state=42
    )
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    
    results[config['name']] = mse
    print(f"{config['name']} (depth {config['min']}-{config['max']}): MSE = {mse:.2f}")

# Visualize
plt.figure(figsize=(10, 5))
plt.bar(results.keys(), results.values(), color='skyblue')
plt.ylabel('Mean Squared Error')
plt.title('Effect of Tree Depth Configuration')
plt.grid(True, alpha=0.3, axis='y')
plt.show()

### GPU Acceleration

For large datasets, you can enable GPU acceleration with CatBoost models.

In [ ]:
# GPU configuration (requires CUDA-enabled GPU)
# model_gpu = RandomCatBoostRegressor(
#     num_iterations=20,
#     learning_rate=0.5,
#     task_type='GPU',  # Enable GPU
#     tree_logging_level='Silent',
#     random_state=42
# )

# For CPU (default)
model_cpu = RandomCatBoostRegressor(
    num_iterations=20,
    learning_rate=0.5,
    task_type='CPU',  # Use CPU
    tree_logging_level='Silent',
    random_state=42
)

print("Models configured for CPU/GPU")

### Regularization

Control overfitting using `tree_reg_lambda` parameter (L2 regularization).

In [ ]:
# Compare different regularization strengths
reg_values = [0.1, 1.0, 3.0, 10.0]
reg_results = {'train': [], 'test': []}

for reg in reg_values:
    model = RandomLGBMRegressor(
        num_iterations=20,
        learning_rate=0.5,
        tree_reg_lambda=reg,
        tree_iterations=50,
        random_state=42
    )
    
    model.fit(X_train, y_train)
    
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    
    train_mse = mean_squared_error(y_train, train_pred)
    test_mse = mean_squared_error(y_test, test_pred)
    
    reg_results['train'].append(train_mse)
    reg_results['test'].append(test_mse)
    
    print(f"Reg λ={reg}: Train MSE={train_mse:.2f}, Test MSE={test_mse:.2f}")

# Visualize
plt.figure(figsize=(10, 5))
plt.plot(reg_values, reg_results['train'], 'o-', label='Train MSE', linewidth=2)
plt.plot(reg_values, reg_results['test'], 's-', label='Test MSE', linewidth=2)
plt.xlabel('Regularization (λ)')
plt.ylabel('Mean Squared Error')
plt.title('Effect of L2 Regularization')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xscale('log')
plt.show()

## 3. Performance Optimization

### Choosing the Right Number of Iterations

More iterations don't always mean better performance. Use early stopping and validation data.

In [ ]:
# Track performance over iterations
from sklearn.metrics import mean_squared_error

X, y = make_regression(n_samples=800, n_features=20, noise=20, random_state=42)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Train model with validation
model = RandomCatBoostRegressor(
    num_iterations=50,
    learning_rate=0.3,
    early_stopping_rounds=10,
    tree_logging_level='Silent',
    random_state=42
)

model.fit(X_train, y_train, X_eval=X_val, y_eval=y_val)

print(f"\nStopped at {len(model.ensemble_)} iterations (max was 50)")

### Balancing Speed vs Accuracy

- **For speed**: Lower `num_iterations`, higher `learning_rate`, lower `tree_iterations`
- **For accuracy**: Higher `num_iterations`, lower `learning_rate`, higher `tree_iterations`

In [ ]:
import time

# Fast configuration
fast_model = RandomLGBMRegressor(
    num_iterations=5,
    learning_rate=1.0,
    tree_iterations=10,
    random_state=42
)

start = time.time()
fast_model.fit(X_train, y_train)
fast_time = time.time() - start
fast_mse = mean_squared_error(y_test, fast_model.predict(X_test))

print(f"Fast config: {fast_time:.2f}s, MSE={fast_mse:.2f}")

# Accurate configuration
accurate_model = RandomLGBMRegressor(
    num_iterations=20,
    learning_rate=0.3,
    tree_iterations=100,
    random_state=42
)

start = time.time()
accurate_model.fit(X_train, y_train)
accurate_time = time.time() - start
accurate_mse = mean_squared_error(y_test, accurate_model.predict(X_test))

print(f"Accurate config: {accurate_time:.2f}s, MSE={accurate_mse:.2f}")
print(f"\nSpeedup: {accurate_time/fast_time:.1f}x")
print(f"MSE improvement: {(fast_mse-accurate_mse)/fast_mse*100:.1f}%")

## 4. Real-World Examples

### Example 1: California Housing Dataset

In [ ]:
# Load California housing dataset
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

# Load data
housing = fetch_california_housing()
X, y = housing.data, housing.target

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

print(f"Dataset shape: {X.shape}")
print(f"Features: {housing.feature_names}")
print(f"Target: Median house value")

In [ ]:
# Train multiple models
models = {
    'RandomLGBM': RandomLGBMRegressor(
        num_iterations=20, learning_rate=0.3, 
        tree_iterations=100, random_state=42
    ),
    'RandomCatBoost': RandomCatBoostRegressor(
        num_iterations=20, learning_rate=0.3,
        tree_iterations=100, tree_logging_level='Silent', random_state=42
    ),
    'RandomMixed': RandomRegressor(
        num_iterations=20, learning_rate=0.3,
        tree_iterations=100, tree_logging_level='Silent', random_state=42
    ),
}

housing_results = {}

for name, model in models.items():
    print(f"\nTraining {name}...")
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    
    from sklearn.metrics import r2_score
    r2 = r2_score(y_test, y_pred)
    
    housing_results[name] = {'RMSE': rmse, 'R2': r2}
    print(f"{name}: RMSE={rmse:.4f}, R²={r2:.4f}")

# Visualize
import pandas as pd
df = pd.DataFrame(housing_results).T
print("\n", df)

### Example 2: Multi-feature Classification

In [ ]:
# Create a complex classification dataset
X, y = make_classification(
    n_samples=2000,
    n_features=30,
    n_informative=20,
    n_redundant=5,
    n_clusters_per_class=3,
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Train classifier
classifier = RandomClassifier(
    loss=LogisticLoss,
    num_iterations=15,
    learning_rate=0.5,
    min_max_depth=4,
    max_max_depth=7,
    tree_iterations=100,
    tree_logging_level='Silent',
    random_state=42
)

print("Training on complex classification task...")
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)
y_proba = classifier.predict_proba(X_test)

# Detailed evaluation
from sklearn.metrics import classification_report, confusion_matrix

print(f"\nAccuracy: {accuracy_score(y_test, y_pred):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
for i in range(2):
    for j in range(2):
        plt.text(j, i, cm[i, j], ha='center', va='center')
plt.show()

## 5. Troubleshooting

### Common Issues and Solutions

#### Issue 1: Model Underfitting

**Symptoms**: High training and test error

**Solutions**:
- Increase `num_iterations`
- Increase tree depth range (`min_max_depth`, `max_max_depth`)
- Decrease `learning_rate` and increase `num_iterations` proportionally
- Increase `tree_iterations` for base learners
- Decrease regularization (`tree_reg_lambda`)

In [ ]:
# Example: Fixing underfitting
X, y = make_regression(n_samples=500, n_features=10, noise=5, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Underfitting model
underfit = RandomLGBMRegressor(
    num_iterations=3,
    learning_rate=1.0,
    min_max_depth=2,
    max_max_depth=2,
    tree_iterations=10,
    random_state=42
)
underfit.fit(X_train, y_train)
print(f"Underfit model MSE: {mean_squared_error(y_test, underfit.predict(X_test)):.2f}")

# Improved model
improved = RandomLGBMRegressor(
    num_iterations=20,
    learning_rate=0.3,
    min_max_depth=4,
    max_max_depth=8,
    tree_iterations=100,
    random_state=42
)
improved.fit(X_train, y_train)
print(f"Improved model MSE: {mean_squared_error(y_test, improved.predict(X_test)):.2f}")

#### Issue 2: Model Overfitting

**Symptoms**: Low training error, high test error

**Solutions**:
- Use early stopping with validation data
- Increase regularization (`tree_reg_lambda`)
- Decrease tree depth range
- Decrease `num_iterations`
- Increase `learning_rate` (with fewer iterations)

In [ ]:
# Example: Preventing overfitting
X, y = make_regression(n_samples=200, n_features=50, noise=10, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

# With regularization and early stopping
regularized = RandomCatBoostRegressor(
    num_iterations=50,
    learning_rate=0.3,
    tree_reg_lambda=5.0,  # Stronger regularization
    early_stopping_rounds=5,  # Early stopping
    tree_logging_level='Silent',
    random_state=42
)

regularized.fit(X_train, y_train, X_eval=X_val, y_eval=y_val)
print(f"Regularized model stopped at {len(regularized.ensemble_)} iterations")

#### Issue 3: Slow Training

**Solutions**:
- Reduce `tree_iterations` for base learners
- Use CPU/GPU appropriately
- Reduce `num_iterations`
- Use simpler models (fewer depth options)
- Increase `learning_rate` and reduce `num_iterations`

In [ ]:
# Fast training configuration
fast_config = {
    'num_iterations': 10,
    'learning_rate': 0.8,
    'min_max_depth': 3,
    'max_max_depth': 5,
    'tree_iterations': 20,
    'random_state': 42
}

fast_model = RandomLGBMRegressor(**fast_config)
print("Fast training configuration:")
print(fast_config)

## Conclusion

This advanced guide covered:

1. Understanding the random boosting algorithm
2. Custom model configurations (tree depth, GPU, regularization)  
3. Performance optimization strategies
4. Real-world dataset examples
5. Troubleshooting common issues

## Bonus: RandomMachine vs Baseline Comparison

**Important question: Is RandomMachine actually better than plain LightGBM/CatBoost?**

Run the comparison below to see for yourself: